In [ ]:
# Modified from
# https://github.com/pytorch/vision/blob/release/0.8.0/torchvision/models/resnet.py

import torch
from torch import Tensor
import torch.nn as nn
from torch.hub import load_state_dict_from_url
from typing import Type, Any, Callable, Union, List, Optional
import os
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

import time
import copy
import numpy as np
import math
from torch.autograd import Variable
from PIL import Image

In [ ]:
def set_random_seeds(random_seed=0):

    torch.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

def prepare_dataloader(num_workers=8, train_batch_size=128, eval_batch_size=256):

    train_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    test_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    train_set = torchvision.datasets.CIFAR10(root="data", train=True, download=True, transform=train_transform) 
    # We will use test set for validation and test in this project.
    # Do not use test set for validation in practice!
    test_set = torchvision.datasets.CIFAR10(root="data", train=False, download=True, transform=test_transform)

    train_sampler = torch.utils.data.RandomSampler(train_set)
    test_sampler = torch.utils.data.SequentialSampler(test_set)

    train_loader = torch.utils.data.DataLoader(
        dataset=train_set, batch_size=train_batch_size,
        sampler=train_sampler, num_workers=num_workers)

    test_loader = torch.utils.data.DataLoader(
        dataset=test_set, batch_size=eval_batch_size,
        sampler=test_sampler, num_workers=num_workers)

    return train_loader, test_loader

In [ ]:
def evaluate_model(model, test_loader, device, criterion=None):

    model.eval()
    model.to(device)

    running_loss = 0
    running_corrects = 0

    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        if criterion is not None:
            loss = criterion(outputs, labels).item()
        else:
            loss = 0

        # statistics
        running_loss += loss * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    eval_loss = running_loss / len(test_loader.dataset)
    eval_accuracy = running_corrects / len(test_loader.dataset)

    return eval_loss, eval_accuracy

In [ ]:
def train_model(model, train_loader, test_loader, device):

    # The training configurations were not carefully selected.
    learning_rate = 1e-4
    num_epochs = 7

    criterion = nn.CrossEntropyLoss()

    model.to(device)

    # It seems that SGD optimizer is better than Adam optimizer for ResNet18 training on CIFAR10.
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-5)
    # optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

    for epoch in range(num_epochs):

        # Training
        model.train()

        running_loss = 0
        running_corrects = 0

        for inputs, labels in train_loader:

            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = running_corrects / len(train_loader.dataset)

        # Evaluation
        model.eval()
        eval_loss, eval_accuracy = evaluate_model(model=model, test_loader=test_loader, device=device, criterion=criterion)

        print("Epoch: {:02d} Train Loss: {:.3f} Train Acc: {:.3f} Eval Loss: {:.3f} Eval Acc: {:.3f}".format(epoch, train_loss, train_accuracy, eval_loss, eval_accuracy))

    return model


In [ ]:
def calibrate_model(model, loader, device=torch.device("cpu:0")):

    model.to(device)
    model.eval()

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        _ = model(inputs)


In [ ]:
def measure_inference_latency(model,
                              device,
                              input_size=(1, 3, 32, 32),
                              num_samples=100,
                              num_warmups=10):

    model.to(device)
    model.eval()

    x = torch.rand(size=input_size).to(device)

    with torch.no_grad():
        for _ in range(num_warmups):
            _ = model(x)
    torch.cuda.synchronize()

    with torch.no_grad():
        start_time = time.time()
        for _ in range(num_samples):
            _ = model(x)
            torch.cuda.synchronize()
        end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ave = elapsed_time / num_samples

    return elapsed_time_ave

In [ ]:
def save_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.save(model.state_dict(), model_filepath)

def load_model(model, model_filepath, device):

    model.load_state_dict(torch.load(model_filepath, map_location=device))

    return model

def save_torchscript_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.jit.save(torch.jit.script(model), model_filepath)

def load_torchscript_model(model_filepath, device):

    model = torch.jit.load(model_filepath, map_location=device)

    return model


In [ ]:
def create_model(num_classes=10):

    # The number of channels in ResNet18 is divisible by 8.
    # This is required for fast GEMM integer matrix multiplication.
    # model = torchvision.models.resnet18(pretrained=False)
    model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
    
    model.classifier[4] = nn.Linear(4096,1024)

    #Updating the third and the last classifier that is the output layer of the network. Make sure to have 10 output nodes if we are going to get 10 class labels through our model.
    model.classifier[6] = nn.Linear(1024,10)
    # We would use the pretrained ResNet18 as a feature extractor.
    # for param in model.parameters():
    #     param.requires_grad = False
    
    # Modify the last FC layer
    # num_features = model.fc.in_features
    # model.fc = nn.Linear(num_features, 10)

    return model


In [ ]:
class QuantizedAlexNet(nn.Module):
    def __init__(self, model_fp32):
        super(QuantizedAlexNet, self).__init__()
        # QuantStub converts tensors from floating point to quantized.
        # This will only be used for inputs.
        self.quant = torch.quantization.QuantStub()
        # DeQuantStub converts tensors from quantized to floating point.
        # This will only be used for outputs.
        self.dequant = torch.quantization.DeQuantStub()
        # FP32 model
        self.model_fp32 = model_fp32

    def forward(self, x):
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.model_fp32(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x

In [ ]:
def model_equivalence(model_1, model_2, device, rtol=1e-05, atol=1e-08, num_tests=100, input_size=(1,3,224,224)):

    model_1.to(device)
    model_2.to(device)

    for _ in range(num_tests):
        x = torch.rand(size=input_size).to(device)
        y1 = model_1(x).detach().cpu().numpy()
        y2 = model_2(x).detach().cpu().numpy()
        if np.allclose(a=y1, b=y2, rtol=rtol, atol=atol, equal_nan=False) == False:
            print("Model equivalence test sample failed: ")
            print(y1)
            print(y2)
            return False

    return True


In [ ]:
random_seed = 0
num_classes = 10
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

model_dir = "saved_models"
model_filename = "alexnet_cifar10.pt"
quantized_model_filename = "alexnet_quantized_cifar10.pt"
model_filepath = os.path.join(model_dir, model_filename)
quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)

set_random_seeds(random_seed=random_seed)

# Create an untrained model.
model = create_model(num_classes=num_classes)

train_loader, test_loader = prepare_dataloader(num_workers=2, train_batch_size=128, eval_batch_size=256)

# Train model.
model = train_model(model=model, train_loader=train_loader, test_loader=test_loader, device=cuda_device)
#model = load_model(model=model, model_filepath="alexnet_cifar10.pt", device=cuda_device)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
Epoch: 00 Train Loss: 1.511 Train Acc: 0.466 Eval Loss: 0.848 Eval Acc: 0.711
Epoch: 01 Train Loss: 0.853 Train Acc: 0.699 Eval Loss: 0.657 Eval Acc: 0.777
Epoch: 02 Train Loss: 0.724 Train Acc: 0.747 Eval Loss: 0.597 Eval Acc: 0.796
Epoch: 03 Train Loss: 0.659 Train Acc: 0.770 Eval Loss: 0.543 Eval Acc: 0.815
Epoch: 04 Train Loss: 0.617 Train Acc: 0.786 Eval Loss: 0.527 Eval Acc: 0.821
Epoch: 05 Train Loss: 0.582 Train Acc: 0.798 Eval Loss: 0.493 Eval Acc: 0.833
Epoch: 06 Train Loss: 0.554 Train Acc: 0.807 Eval Loss: 0.483 Eval Acc: 0.837


In [ ]:
#import os
isFile = os.path.isfile("readme.txt")
print(isFile)
with open("readme.txt","w") as f:
    f.close()
isFile = os.path.isfile("readme.txt")
print(isFile)

False
True


In [ ]:
isFile = False
features = []
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 10, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes),
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if(isFile):
            feature1 = x.reshape(x.shape[0], -1)
            """
            with open('readme.txt', 'a') as f:
                for i in range(x.size(dim=0)):
                    for j in range(x.size(dim=1)):
                        for k in range(x.size(dim=2)):
                            for l in range(x.size(dim=3)):
                                f.write('{}\t'.format(x[i][j][k][l]))
                f.write("\n\n")
            """
            x = self.features[0](x)
            """
            with open('readme.txt', 'a') as f:
                for i in range(x.size(dim=0)):
                    for j in range(x.size(dim=1)):
                        for k in range(x.size(dim=2)):
                            for l in range(x.size(dim=3)):
                                f.write('{}\t'.format(x[i][j][k][l]))
                f.write("\n\n")
            """
            feature2 = x.reshape(x.shape[0], -1)
            x = self.features[1](x)
            x = self.features[2](x)
            """
            with open('readme.txt', 'a') as f:
                for i in range(x.size(dim=0)):
                    for j in range(x.size(dim=1)):
                        for k in range(x.size(dim=2)):
                            for l in range(x.size(dim=3)):
                                f.write('{}\t'.format(x[i][j][k][l]))
                f.write("\n\n")
            """
            feature3 = x.reshape(x.shape[0], -1)
            x = self.features[3](x)
            """
            with open('readme.txt', 'a') as f:
                for i in range(x.size(dim=0)):
                    for j in range(x.size(dim=1)):
                        for k in range(x.size(dim=2)):
                            for l in range(x.size(dim=3)):
                                f.write('{}\t'.format(x[i][j][k][l]))
                f.write("\n\n")
            """
            feature4 = x.reshape(x.shape[0], -1)
            x = self.features[4](x)
            x = self.features[5](x)
            """
            with open('readme.txt', 'a') as f:
                for i in range(x.size(dim=0)):
                    for j in range(x.size(dim=1)):
                        for k in range(x.size(dim=2)):
                            for l in range(x.size(dim=3)):
                                f.write('{}\t'.format(x[i][j][k][l]))
                f.write("\n\n")
            """
            feature5 = x.reshape(x.shape[0], -1)
            x = self.features[6](x)
            """
            with open('readme.txt', 'a') as f:
                for i in range(x.size(dim=0)):
                    for j in range(x.size(dim=1)):
                        for k in range(x.size(dim=2)):
                            for l in range(x.size(dim=3)):
                                f.write('{}\t'.format(x[i][j][k][l]))
                f.write("\n\n")
            """
            feature6 = x.reshape(x.shape[0], -1)
            x = self.features[7](x)
            """
            with open('readme.txt', 'a') as f:
                for i in range(x.size(dim=0)):
                    for j in range(x.size(dim=1)):
                        for k in range(x.size(dim=2)):
                            for l in range(x.size(dim=3)):
                                f.write('{}\t'.format(x[i][j][k][l]))
                f.write("\n\n")
            """
            feature7 = x.reshape(x.shape[0], -1)
            x = self.features[8](x)
            """
            with open('readme.txt', 'a') as f:
                for i in range(x.size(dim=0)):
                    for j in range(x.size(dim=1)):
                        for k in range(x.size(dim=2)):
                            for l in range(x.size(dim=3)):
                                f.write('{}\t'.format(x[i][j][k][l]))
                f.write("\n\n")
            """
            feature8 = x.reshape(x.shape[0], -1)
            x = self.features[9](x)
            """
            with open('readme.txt', 'a') as f:
                for i in range(x.size(dim=0)):
                    for j in range(x.size(dim=1)):
                        for k in range(x.size(dim=2)):
                            for l in range(x.size(dim=3)):
                                f.write('{}\t'.format(x[i][j][k][l]))
                f.write("\n\n")
            """
            feature9 = x.reshape(x.shape[0], -1)
            x = self.features[10](x)

            """with open('readme.txt', 'a') as f:
                for i in range(x.size(dim=0)):
                    for j in range(x.size(dim=1)):
                        for k in range(x.size(dim=2)):
                            for l in range(x.size(dim=3)):
                                f.write('{}\t'.format(x[i][j][k][l]))
                f.write("\n\n")
            """
            feature10 = x.reshape(x.shape[0], -1)
            x = self.features[11](x)
            x = self.features[12](x)      
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.classifier[0](x)
            x = self.classifier[1](x)
            x = self.classifier[2](x)
            x = self.classifier[3](x)
            x = self.classifier[4](x)
            x = self.classifier[5](x)
            x = self.classifier[6](x)
            #feature = torch.cat((feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8, feature9, feature10),0)
            print(feature2)
            feature = []
            feature.append(feature1)
            feature.append(feature2)
            feature.append(feature3)
            feature.append(feature4)
            feature.append(feature5)
            feature.append(feature6)
            feature.append(feature7)
            feature.append(feature8)
            feature.append(feature9)
            feature.append(feature10)
            for i in range(10):
                features.append(feature[i])
            return x
        else:
            x = self.features[0](x)
            x = self.features[1](x)
            x = self.features[2](x)
            x = self.features[3](x)
            x = self.features[4](x)
            x = self.features[5](x)
            x = self.features[6](x)
            x = self.features[7](x)
            x = self.features[8](x)
            x = self.features[9](x)
            x = self.features[10](x)
            x = self.features[11](x)
            x = self.features[12](x)      
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.classifier[0](x)
            x = self.classifier[1](x)
            x = self.classifier[2](x)
            x = self.classifier[3](x)
            x = self.classifier[4](x)
            x = self.classifier[5](x)
            x = self.classifier[6](x)
            return x
model2 = AlexNet()
print(model2)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
model2.load_state_dict(model.state_dict())
_, fp32_eval_accuracy = evaluate_model(model=model2, test_loader=test_loader, device=cpu_device, criterion=None)
print("FP32 evaluation accuracy: {:.3f}".format(fp32_eval_accuracy))

In [ ]:
isFile = True

"""
with open("readme.txt", "w") as f:
  f.write("")
  f.close()
"""
imsize = 224
loader = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])


def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image  #assumes that you're using GPU

image = image_loader("images.jpg")
"""
print(image)
with open("readme.txt", "a") as f:
  f.write('{} '.format(image[0][0][0][0]))"""
out, feature = model2(image)
isFile = False
print(out)
print(feature[9])

In [ ]:
with open("float32_input&output.txt", "w") as f:
    for i in range(10):
        f.write("{}\n".format(i+1))
        f.write("{}".format(feature[i]))
        f.write("\n")
    

In [ ]:
print(image)

In [ ]:
for param_tensor in model.state_dict():
    print(param_tensor, "\n", model.state_dict()[param_tensor])
    model2.state_dict()[param_tensor] = model.state_dict()[param_tensor]

In [ ]:
for param_tensor in model2.state_dict():
    print(param_tensor, "\n", model2.state_dict()[param_tensor])

In [ ]:
# Save model.
save_model(model=model2, model_dir=model_dir, model_filename=model_filename)
# Load a pretrained model.
model2 = load_model(model=model2, model_filepath=model_filepath, device=cuda_device)
# Move the model to CPU since static quantization does not support CUDA currently.
model2.to(cpu_device)

In [ ]:

# Make a copy of the model for layer fusion
fused_model = copy.deepcopy(model2)

model2.eval()
# The model has to be switched to evaluation mode before any layer fusion.
# Otherwise the quantization will not work correctly.
fused_model.eval()

# Fuse the model in place rather manually.
#fused_model = torch.quantization.fuse_modules(fused_model, [["relu"]], inplace=True)
for module_name, module in fused_model.named_children():
    if "layer" in module_name:
        for basic_block_name, basic_block in module.named_children():
            torch.quantization.fuse_modules(basic_block, [["conv1", "bn1", "relu1"], ["conv2", "bn2"]], inplace=True)
            for sub_block_name, sub_block in basic_block.named_children():
                if sub_block_name == "downsample":
                    torch.quantization.fuse_modules(sub_block, [["0", "1"]], inplace=True)

# Print FP32 model.
print(model2)
# Print fused model.
print(fused_model)

# Model and fused model should be equivalent.
assert model_equivalence(model_1=model2, model_2=fused_model, device=cpu_device, rtol=1e-03, atol=1e-06, num_tests=100, input_size=(1,3,224,224)), "Fused model is not equivalent to the original model!"


In [ ]:

# Prepare the model for static quantization. This inserts observers in
# the model that will observe activation tensors during calibration.
quantized_model = QuantizedAlexNet(model_fp32=fused_model)
# Using un-fused model will fail.
# Because there is no quantized layer implementation for a single batch normalization layer.
# quantized_model = QuantizedResNet18(model_fp32=model)
# Select quantization schemes from 
# https://pytorch.org/docs/stable/quantization-support.html
quantization_config = torch.quantization.get_default_qconfig("fbgemm")
# Custom quantization configurations
# quantization_config = torch.quantization.default_qconfig

quantized_model.qconfig = quantization_config
# Print quantization configurations
#print(quantized_model.qconfig)
print(quantized_model)

torch.quantization.prepare(quantized_model, inplace=True)


# Use training data for calibration.
calibrate_model(model=quantized_model, loader=train_loader, device=cpu_device)

quantized_model = torch.quantization.convert(quantized_model, inplace=True)

# Using high-level static quantization wrapper
# The above steps, including torch.quantization.prepare, calibrate_model, and torch.quantization.convert, are also equivalent to
# quantized_model = torch.quantization.quantize(model=quantized_model, run_fn=calibrate_model, run_args=[train_loader], mapping=None, inplace=False)

quantized_model.eval()

# Print quantized model.
print(quantized_model)


In [ ]:
quantized_model.model_fp32.features[0].scale = math.pow(2, round(math.log(quantized_model.model_fp32.features[0].scale,2), 0))
print(quantized_model.model_fp32.features[0].scale)

quantized_model.model_fp32.features[3].scale = math.pow(2, round(math.log(quantized_model.model_fp32.features[3].scale,2), 0))
print(quantized_model.model_fp32.features[3].scale)

quantized_model.model_fp32.features[6].scale = math.pow(2, round(math.log(quantized_model.model_fp32.features[6].scale,2), 0))
print(quantized_model.model_fp32.features[6].scale)

quantized_model.model_fp32.features[8].scale = math.pow(2, round(math.log(quantized_model.model_fp32.features[8].scale,2), 0))
print(quantized_model.model_fp32.features[8].scale)

quantized_model.model_fp32.features[10].scale = math.pow(2, round(math.log(quantized_model.model_fp32.features[10].scale,2), 0))
print(quantized_model.model_fp32.features[10].scale)

quantized_model.model_fp32.classifier[1].scale = math.pow(2, round(math.log(quantized_model.model_fp32.classifier[1].scale,2), 0))
print(quantized_model.model_fp32.classifier[1].scale)

quantized_model.model_fp32.classifier[4].scale = math.pow(2, round(math.log(quantized_model.model_fp32.classifier[4].scale,2), 0))
print(quantized_model.model_fp32.classifier[4].scale)

quantized_model.model_fp32.classifier[6].scale = math.pow(2, round(math.log(quantized_model.model_fp32.classifier[6].scale,2), 0))
print(quantized_model.model_fp32.classifier[6].scale)

quantized_model.quant.scale[0]=math.pow(2, round( math.log(quantized_model.quant.scale[0],2),0))
print(quantized_model)

In [ ]:
# Save quantized model.
save_torchscript_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename)

# Load quantized model.
quantized_jit_model = load_torchscript_model(model_filepath=quantized_model_filepath, device=cpu_device)
quantized_jit_model 

In [ ]:
_, fp32_eval_accuracy = evaluate_model(model=model2, test_loader=test_loader, device=cpu_device, criterion=None)
_, int16_eval_accuracy = evaluate_model(model=quantized_model, test_loader=test_loader, device=cpu_device, criterion=None)

# Skip this assertion since the values might deviate a lot.
# assert model_equivalence(model_1=model, model_2=quantized_jit_model, device=cpu_device, rtol=1e-01, atol=1e-02, num_tests=100, input_size=(1,3,32,32)), "Quantized model deviates from the original model too much!"

print("FP32 evaluation accuracy: {:.3f}".format(fp32_eval_accuracy))
print("INT8 evaluation accuracy: {:.3f}".format(int16_eval_accuracy))



In [ ]:
from torch.autograd import Variable
from PIL import Image
inference = True
imsize = 224
loader = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])


def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)
    return image

image = image_loader("images.jpg")

output= quantized_model(image)
print(output)

In [ ]:
torch.set_printoptions(threshold=10_000000)
np.set_printoptions(threshold=10_000000)
max = -999999
min = 999999

with open("quantized_input&output.txt", "w") as f:
    for index in range(10):
        f.write("{}\n".format(features[index][0].size(dim=0)))
        f.write("{}\n".format(features[index][0]))
        for i in range(features[index][0].size(dim=0)):
            if(features[index][0][i] > max):
                max = features[index][0][i]
            if(features[index][0][i] < min):
                min = features[index][0][i]

print(max)
print(min)

In [ ]:
torch.set_printoptions(threshold=10_00000)
np.set_printoptions(threshold=10_00000)
max = -99999
min = 99999
for index in range(10):
    for i in range(features[index][0].size(dim=0)):
        if(features[index][0][i] > max):
          max = features[index][0][i]
        if(features[index][0][i] < min):
          min = features[index][0][i]

print(max)
print(min)

In [ ]:
fp32_cpu_inference_latency = measure_inference_latency(model=model, device=cpu_device, input_size=(1,3,224,224), num_samples=100)
int16_cpu_inference_latency = measure_inference_latency(model=quantized_model, device=cpu_device, input_size=(1,3,224,224), num_samples=100)
int16_jit_cpu_inference_latency = measure_inference_latency(model=quantized_jit_model, device=cpu_device, input_size=(1,3,224,224), num_samples=100)
fp32_gpu_inference_latency = measure_inference_latency(model=model, device=cuda_device, input_size=(1,3,224,224), num_samples=100)

print("FP32 CPU Inference Latency: {:.2f} ms / sample".format(fp32_cpu_inference_latency * 1000))
#print("FP32 CUDA Inference Latency: {:.2f} ms / sample".format(fp32_gpu_inference_latency * 1000))
print("INT16 CPU Inference Latency: {:.2f} ms / sample".format(int16_cpu_inference_latency * 1000))
#print("INT16 JIT CPU Inference Latency: {:.2f} ms / sample".format(int16_jit_cpu_inference_latency * 1000))

In [ ]:
with open("quantized_weight.txt", "w") as f:
  for weight in quantized_model.state_dict():
    np.set_printoptions(threshold=10_0000)
    f.write(weight)
    f.write('\n')
    f.write("{}".format(quantized_model.state_dict()[weight]))

In [ ]:
with open("quantized_weight.txt", "w") as f:
  for weight in quantized_model.state_dict():
      np.set_printoptions(suppress=True)
      torch.set_printoptions(threshold=10_000000)
      if(type(quantized_model.state_dict()[weight]) == torch.Tensor):
          f.write("{}\n".format(quantized_model.state_dict()[weight].size()))
          f.write("{}".format(quantized_model.state_dict()[weight]))
          f.write("\n")

In [ ]:
with open("float_weight.txt", "w") as f:
    for param_tensor in model2.state_dict():
        f.write("{}\n".format(param_tensor))
        f.write("{}".format(model2.state_dict()[param_tensor].size()))
        f.write("{}\n".format(model2.state_dict()[param_tensor]))
        #print(param_tensor, "\n", model2.state_dict()[param_tensor])

In [ ]:
for param in model2.parameters():
    #torch.set_printoptions(threshold=10_0000)
    print(param.data.size())
    print(param.data)

In [ ]:
"""quantization_config = torch.quantization.QConfig(
    activation=torch.quantization.MinMaxObserver.with_args(
        dtype=torch.quint8,
        qscheme=torch.per_tensor_symmetric,
    ), 
    weight=torch.quantization.MinMaxObserver.with_args(
        dtype=torch.qint8, 
        qscheme=torch.per_tensor_symmetric,
    )
)
FP32 evaluation accuracy: 0.854
INT8 evaluation accuracy: 0.814
"""
"""
quantization_config = torch.quantization.get_default_qconfig("fbgemm")
FP32 evaluation accuracy: 0.850
INT8 evaluation accuracy: 0.848
"""

In [ ]:
for param_tensor in model2.state_dict():
    print(param_tensor, "\n", model2.state_dict()[param_tensor])

In [ ]:
for param_tensor in fused_model.state_dict():
    print(param_tensor, "\n", fused_model.state_dict()[param_tensor])

In [ ]:
quantized_model.model_fp32.features[0].scale = 1


In [ ]:
quantized_model.model_fp32.features[0].scale = 0.25
for param_tensor in quantized_model.state_dict():
    print(param_tensor, "\n", quantized_model.state_dict()[param_tensor])